In [ ]:
import pandas as pd
import numpy as np
import datetime as dt


# **Task** **1**

In [ ]:
#dummy data for task 1

df = pd.read_csv('data.csv')
df = df.drop([1693,1694],axis=0)

In [ ]:

def date_difference(dataframe):

  #   Note***/// This function will do the job for any format of date values in column however i tried but cannot figure out all possible ways to 
  #   filter the date irrespective of known formats from text data(as in row 1693 and 1694 in dummy data) because of many format possible and process is too time consuming

    #drop columns if they have empty values more than 30% of total data otherwise drop the rows ith missing values
    for col in dataframe.columns:
       if dataframe[col].isnull().sum()>(dataframe[col].size)*0.3:
           dataframe = dataframe.drop(col,axis=1)
    dataframe.dropna()

    #converting columns with date into datetime values
    dataframe = dataframe.apply(lambda col: pd.to_datetime(col,errors='ignore', exact=False,utc=True)
                  if col.dtype== 'object'
                  else col,
                    axis=0 )
    
    #used for finding colummns with datetime values
    a=[i for i in dataframe.columns if str(dataframe[i].dtypes)[:10] == 'datetime64']

    #dropping columns with current timestamp due to error
    for n,i in enumerate(a):
      if dataframe[i][0] == dt.date.today():
        dataframe = dataframe.drop(i,axis=1)
        a.pop(n) 

    #taking only date from the timestamp columns if any
    for i in a:
      dataframe[i]= dataframe[i].dt.date
    
    # find the difference between the dates as given and prohibit formation of multiple columns with same values
    for i in range(len(a)-1):
      dataframe["days({}-{})".format(a[i],a[i+1])] = dataframe[a[i]]- dataframe[a[i+1]]
      dataframe["days({}-{})".format(a[i],a[i+1])] = dataframe["days({}-{})".format(a[i],a[i+1])].apply(lambda x: x.days)
    if len(a)>2:
      dataframe["days({}-{})".format(a[0],a[len(a)-1])] = dataframe[a[0]]-dataframe[a[-1]] 
      dataframe["days({}-{})".format(a[0],a[len(a)-1])]  =dataframe["days({}-{})".format(a[0],a[len(a)-1])].apply(lambda x: x.days)

    return dataframe

In [ ]:
date_difference(df)


,Unnamed: 0,date,id,country_name,Date,Depth,Type,days(date-Date)
0,0,2007-03-02,34,United States,1965-01-02,131.6,Earthquake,15399.0
1,1,2007-03-22,42,United States,1965-01-04,80.0,Earthquake,15417.0
2,2,2007-04-06,56,United States,1965-01-05,20.0,Earthquake,15431.0
3,3,2007-04-14,59,Canada,1965-01-08,15.0,Earthquake,15436.0
4,4,2007-04-15,61,United States,1965-01-09,15.0,Earthquake,15436.0
...,...,...,...,...,...,...,...,...
1688,1688,2015-12-07,7535,United States,1970-08-29,15.0,Earthquake,16536.0
1689,1689,2016-02-22,7537,United States,1970-08-30,35.0,Earthquake,16612.0
1690,1690,2016-02-23,7539,United States,1970-08-30,647.6,Earthquake,16613.0
1691,1691,2016-02-26,7540,United States,1970-09-01,25.0,Earthquake,16614.0


# **Task 2**

In [ ]:
# dummy data for outlier detection

df = pd.DataFrame(np.linspace(500,1500,50))
df.rename(columns={0:'Salary'},inplace=True)
df1 = pd.DataFrame([5000.0,25000.0,1700.0,500000.5])
#df1.rename(columns={0:'Salary'},inplace=True)
data = df['Salary'].append(df1,ignore_index=True)

In [ ]:
#method  (IQR)

def remove_outliers(df1):
    # remove the null values columns if exceeds threshold of 40%.
    for col in df1.columns:
      if df1[col].isnull().sum()>(df1[col].size)*0.4:
          df1 = df1.drop(col,axis=1)
    #otherwise drop rows with null values
    df1.dropna(inplace=True)
    
    #start outlier detection
    columns = list(data.columns)
    num_columns = []
    outliers = []
    for col in columns:
        if data[col].dtype =='float64':
          #now i use IQR method to detect outliers and replace them with 'NaN' values
          b = sorted(data[col])
          q1, q3= np.percentile(b,[25,75])
          iqr = q3-q1
          lower_limit = q1 -(1.5 * iqr) 
          upper_limit = q3 +(1.5 * iqr)
        
          for y in data[col]: 
            if lower_limit<=y<=upper_limit:
                continue
            else:
                outliers.append(y)
                num_columns.append(col)
                data[col].replace({y:np.nan},inplace=True)
    # drop the row having outliers if the total outliers are below 30% of the size of numerical columns having outliers
    if len(outliers)< (data[num_columns].size)*0.3:
      data.dropna(inplace=True)
    # else replace the outlier values with mean
    else:
      for i in num_columns:
        data[i] = data[i].fillna(float(data[i].dropna().mean()))

    return data

In [ ]:
remove_outliers(data)

,0
0,500.000000
1,520.408163
2,540.816327
3,561.224490
4,581.632653
5,602.040816
6,622.448980
7,642.857143
8,663.265306
9,683.673469


# **Task 3**

In [ ]:
arr  = np.linspace(500,1500,50).reshape(10,5)
df = pd.DataFrame(arr)
df

,0,1,2,3,4
0,500.000000,520.408163,540.816327,561.224490,581.632653
1,602.040816,622.448980,642.857143,663.265306,683.673469
2,704.081633,724.489796,744.897959,765.306122,785.714286
3,806.122449,826.530612,846.938776,867.346939,887.755102
4,908.163265,928.571429,948.979592,969.387755,989.795918
5,1010.204082,1030.612245,1051.020408,1071.428571,1091.836735
6,1112.244898,1132.653061,1153.061224,1173.469388,1193.877551
7,1214.285714,1234.693878,1255.102041,1275.510204,1295.918367
8,1316.326531,1336.734694,1357.142857,1377.551020,1397.959184
9,1418.367347,1438.775510,1459.183673,1479.591837,1500.000000


In [ ]:
def remove_high_corr_columns(df1):
    # remove the null values columns if exceeds threshold of 40%.
    # Using this method minimum number of columns are dropped.
    for col in df1.columns:
      if df1[col].isnull().sum()>(df1[col].size)*0.4:
          df1 = df1.drop(col,axis=1)

    #drop rows with null values
    df1.dropna(inplace=True)

    #find the pearson correlation metrics
    corr_matrix = df1.corr().abs()

    #select the upper triangular metrics for singularity of values
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    
    # Find features with correlation greater than threshold
    to_drop = [column for column in upper.columns if any(upper[column] > 0.85)]

    # Drop columns with high pearson correlation 
    df1 = df1.drop(to_drop, axis=1)

    #return the new dataframe
    return df1

In [ ]:
remove_high_corr_columns(df)

,0
0,500.000000
1,602.040816
2,704.081633
3,806.122449
4,908.163265
5,1010.204082
6,1112.244898
7,1214.285714
8,1316.326531
9,1418.367347
